In [1]:
import json, os
import shutil
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def preprocess_tcga_data(json_file, source_dirctory, target_dirctory):

    if not json_file or json_file == '':
        return False
    if not os.path.exists(source_dirctory):
        return False
    if not os.path.exists(target_dirctory):
        os.makedirs(target_dirctory)

    # read json_file
    with open(json_file, 'r') as jf:
        json_data = jf.read()

    json_info = json.loads(json_data)

    for file_info in json_info:
        file_id = file_info['file_id']
        file_name = file_info['file_name']
        submit_id = file_info['associated_entities'][0]['entity_submitter_id']
        source_file = source_dirctory + '/' + file_id + '/' + file_name
        target_file = target_dirctory + '/' + submit_id 
        shutil.copy(source_file, target_file)
        print('file_name :{} copy done'.format(file_name))

preprocess_tcga_data("data/metadata.cart.2023-03-02.json","data","processdata")

In [7]:
import os
#remove unecessary data 
def prodata(source_dirctory, target_dirctory):
    if not os.path.exists(source_dirctory):
        return False
    if not os.path.exists(target_dirctory):
        os.makedirs(target_dirctory)
    files = os.listdir(source_dirctory)

    for file in files:
            file_path = os.path.join(source_dirctory, file)
            df = pd.read_csv(file_path, sep='\t', header=None, skiprows=[
                             0,1,2, 3, 4, 5], usecols=[0,3])
            f = os.path.splitext(file)[0]
            df.to_csv(target_dirctory+'/'+f,sep='\t',index=False, header=False)
            # print(f," done")

prodata("processdata","Process1data")




In [4]:
import pandas as pd
#collect all genes information from gtf file
GFF3 = pd.read_csv(
    filepath_or_buffer='Homo_sapiens.GRCh38.103.chr.gtf',
    sep='\t',
    header=None,
    names=['seqid', 'source', 'type', 'start', 'end',
           'score', 'strand', 'phase', 'attributes'],
    skiprows=[0, 1, 2, 3, 4]
)
GFF3 = GFF3[GFF3['source'].notnull()]

GFF3 = GFF3[GFF3['type'] == 'gene']

GFF3['gene_id'] = GFF3.apply(lambda x: x.attributes.split(
    ';')[0].strip().split(' ')[1].strip('"'), axis=1)
GFF3['gene_name'] = GFF3.apply(lambda x: x.attributes.split(
    ';')[2].strip().split(' ')[1].strip('"'), axis=1)
GFF3['gene_biotype'] = GFF3.apply(lambda x: x.attributes.split(
    ';')[4].strip().split(' ')[1].strip('"'), axis=1)

GFF3 = pd.DataFrame(GFF3, columns=['gene_id', 'gene_name', 'gene_biotype'])
GFF3

C:\Users\Akash Yadav\AppData\Local\Temp\ipykernel_3656\3245440399.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  GFF3 = pd.read_csv(


,gene_id,gene_name,gene_biotype
0,ENSG00000223972,DDX11L1,transcribed_unprocessed_pseudogene
12,ENSG00000227232,WASH7P,unprocessed_pseudogene
25,ENSG00000278267,MIR6859-1,miRNA
28,ENSG00000243485,MIR1302-2HG,lncRNA
36,ENSG00000284332,MIR1302-2,miRNA
...,...,...,...
3073626,ENSG00000198695,MT-ND6,protein_coding
3073632,ENSG00000210194,MT-TE,Mt_tRNA
3073635,ENSG00000198727,MT-CYB,protein_coding
3073640,ENSG00000210195,MT-TT,Mt_tRNA


In [6]:
# cv=0
# for file_name in os.listdir('D:\BTp\Process1data'):
#     if file_name.endswith('.csv'):
#         cv+=1
#     else :
#         file_path = os.path.join('D:\BTp\Process1data', file_name)
#         df = pd.read_csv(file_path, header=None,skiprows=1)
#         df.drop(df.columns[[1,2]], axis=1, inplace=True)
#         df.to_csv(file_path, header=False, index=False)

In [1]:
def resd(filename):
    df = pd.read_csv('./Process1data/'+filename, sep='\t', header=None, names=['gene_id', filename])

    df['gene_id'] = df.apply(lambda x : x.gene_id.split('.')[0], axis = 1)

    x = pd.merge(GFF3, df, on = 'gene_id')

    x.to_csv('./MergeHelp/'+filename+'.csv')
    

In [2]:
import os
def mergeAll(target_df):
    info_files = os.listdir('./Process1data')
    if len(info_files) == 0:
        return None
    for file in info_files:
            resd(file)

In [5]:
mergeAll(GFF3)

In [ ]:
# import warnings
# warnings.filterwarnings("ignore")
# info_files = os.listdir('./MergeHelp')
# new_df= pd.DataFrame()
# for count,file in enumerate(info_files):
#     df= pd.read_csv('./MergeHelp/'+file,sep=',',index_col=False)
#     new_df = new_df.append(df)
#     if count==30 :
#         new_df.to_csv('kk.csv')
#     print(count)
# print(new_df)

In [ ]:

# info_files = os.listdir('./MergeHelp')
# chunks = list()
# for i in range(0,len(info_files),30):
#     chunks.append(info_files[i:i+30])

# for i in range(0,len(chunks),1):
#     merge(chunks[i],i)


In [ ]:
df = pd.read_csv('./MergeHelp/TCGA-05-4244-01A-01R-1107-07.csv')
df.drop('TCGA-05-4244-01A-01R-1107-07',axis=1)
info_files = os.listdir('./MergeHelp')
info_files[0]
for count,filename in enumerate(info_files):
    new_df= pd.read_csv('./MergeHelp/'+filename)
    df[filename.split('.')[0]]= new_df[filename.split('.')[0]]
    print(count)

In [11]:

df.drop('Unnamed: 0', axis=1, inplace=True)
df.to_csv('merged.csv',index=False)

In [9]:
#filter only protein coding genes
df=pd.read_csv('merged.csv',index_col=False)
df = df[df['gene_biotype'] == 'protein_coding']
df

,gene_id,gene_name,gene_biotype,TCGA-05-4244-01A-01R-1107-07,TCGA-05-4390-01A-02R-1755-07,TCGA-05-4395-01A-01R-1206-07,TCGA-05-4396-01A-21R-1858-07,TCGA-05-4397-01A-01R-1206-07,TCGA-05-4398-01A-01R-1206-07,TCGA-05-4402-01A-01R-1206-07,...,TCGA-91-A4BC-01A-11R-A24H-07,TCGA-95-7947-01A-11R-2187-07,TCGA-97-7941-01A-11R-2187-07,TCGA-97-A4M3-01A-11R-A24X-07,TCGA-J2-8194-01A-11R-2241-07,TCGA-L9-A7SV-01A-11R-A39D-07,TCGA-MP-A4T8-01A-11R-A24X-07,TCGA-44-2665-01B-06R-A277-07,TCGA-55-6981-11A-01R-1949-07,TCGA-78-7160-01A-11R-2039-07
8,ENSG00000186092,OR4F5,protein_coding,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,ENSG00000284733,OR4F29,protein_coding,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42,ENSG00000284662,OR4F16,protein_coding,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,ENSG00000187634,SAMD11,protein_coding,86,40,60,106,26,297,463,...,196,33,523,351,133,20,818,7,86,100
62,ENSG00000188976,NOC2L,protein_coding,2771,2690,10741,1418,3792,13909,13680,...,3059,4988,5466,4748,5695,9577,10550,145,1292,2046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60622,ENSG00000212907,MT-ND4L,protein_coding,23241,7977,17037,9814,38474,37573,26784,...,34390,31814,43174,20048,12101,82654,25941,4824,10936,18990
60623,ENSG00000198886,MT-ND4,protein_coding,285747,106073,227894,161413,543519,436951,443771,...,701785,358957,653858,465620,125161,929094,588972,46060,169645,307834
60627,ENSG00000198786,MT-ND5,protein_coding,118522,40225,78042,65764,103032,167032,216614,...,115109,140711,345371,129551,33203,257838,113166,26554,95249,31482
60628,ENSG00000198695,MT-ND6,protein_coding,34417,16242,30793,28412,34233,49022,99245,...,30209,41774,131574,38750,9582,85728,40446,4045,43719,9645


In [13]:
# df.drop(['gene_id','gene_biotype'],axis=1)
df.drop(df.iloc[:, 2:4], inplace=True, axis=1)
df.drop('gene_id',axis=1,inplace=True)
df

,gene_name,TCGA-05-4396-01A-21R-1858-07,TCGA-05-4405-01A-21R-1858-07,TCGA-05-4410-01A-21R-1858-07,TCGA-05-4415-01A-22R-1858-07,TCGA-05-4417-01A-22R-1858-07,TCGA-05-4418-01A-01R-1206-07,TCGA-05-4420-01A-01R-1206-07,TCGA-05-4424-01A-22R-1858-07,TCGA-05-4425-01A-01R-1755-07,...,TCGA-91-A4BC-01A-11R-A24H-07,TCGA-95-7947-01A-11R-2187-07,TCGA-97-7941-01A-11R-2187-07,TCGA-97-A4M3-01A-11R-A24X-07,TCGA-J2-8194-01A-11R-2241-07,TCGA-L9-A7SV-01A-11R-A39D-07,TCGA-MP-A4T8-01A-11R-A24X-07,TCGA-44-2665-01B-06R-A277-07,TCGA-55-6981-11A-01R-1949-07,TCGA-78-7160-01A-11R-2039-07
8,OR4F5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,OR4F29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42,OR4F16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,SAMD11,106,258,50,1217,75,645,156,80,161,...,196,33,523,351,133,20,818,7,86,100
62,NOC2L,1418,2247,2406,7965,1900,8156,7573,2944,2422,...,3059,4988,5466,4748,5695,9577,10550,145,1292,2046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60622,MT-ND4L,9814,11055,19806,20018,16561,22739,46466,12649,4678,...,34390,31814,43174,20048,12101,82654,25941,4824,10936,18990
60623,MT-ND4,161413,191786,296572,210681,194008,392670,534447,225274,52471,...,701785,358957,653858,465620,125161,929094,588972,46060,169645,307834
60627,MT-ND5,65764,61927,195265,46187,43740,112038,53011,95816,15915,...,115109,140711,345371,129551,33203,257838,113166,26554,95249,31482
60628,MT-ND6,28412,29559,78950,11784,13695,41129,11024,38327,6250,...,30209,41774,131574,38750,9582,85728,40446,4045,43719,9645


In [16]:
df.to_csv('FinalData.csv',index=False)

In [12]:
import json


with open('metadata.cart.2023-03-02.json') as f:
    data = json.load(f)

results = []
for obj in data:
    entity_submitter_id = obj['associated_entities'][0]['entity_submitter_id']
    file_id = obj['file_id']
    results.append({'entity_submitter_id': entity_submitter_id, 'file_id': file_id})

print(results)


[{'entity_submitter_id': 'TCGA-Q1-A73Q-01A-21R-A32P-07', 'file_id': '59f827f3-24f7-434a-a43e-b1567b13b743'}, {'entity_submitter_id': 'TCGA-78-7156-01A-11R-2039-07', 'file_id': '129f7ee5-6342-4a24-9317-d52462ce6a70'}, {'entity_submitter_id': 'TCGA-44-6774-01A-21R-1858-07', 'file_id': '3cd897bd-893e-4497-b9a5-56141816cff4'}, {'entity_submitter_id': 'TCGA-91-6829-11A-01R-1858-07', 'file_id': '3f441cc8-98bd-4650-b5ad-19ae8542b666'}, {'entity_submitter_id': 'TCGA-69-A59K-01A-11R-A262-07', 'file_id': '36432508-6673-489b-aefe-d172b83abac2'}, {'entity_submitter_id': 'TCGA-44-2665-01A-01R-A278-07', 'file_id': 'e8da0b02-8b75-4a85-b91a-6f8d1582a95c'}, {'entity_submitter_id': 'TCGA-55-A490-01A-11R-A466-07', 'file_id': '864c2830-c699-433c-bcba-20f0c692854a'}, {'entity_submitter_id': 'TCGA-55-7816-01A-11R-2170-07', 'file_id': '215f38d1-a17c-4189-97f6-ae20361b0f3f'}, {'entity_submitter_id': 'TCGA-44-6147-01B-06R-A277-07', 'file_id': '9a6d8661-241e-4a3e-a6fa-e2eb1962b758'}, {'entity_submitter_id': 'TC

In [ ]:
# import requests
import requests

file_info = results
tu_df= pd.DataFrame()
for info in file_info:
    file_id = info['file_id']
    response = requests.get(f'https://api.gdc.cancer.gov/files/{file_id}?expand=cases.samples')
    data = response.json()['data']
    sample_type = data['cases'][0]['samples'][0]['sample_type']
    print(f'File ID: {file_id}, Sample Type: {sample_type}')
    info['sample_type'] = sample_type
    tu_df = tu_df.append(info,ignore_index=True)
tu_df.drop(columns=['file_id'])



In [14]:
tu_df['sample_type'].value_counts()

Primary Tumor          540
Solid Tissue Normal     59
Recurrent Tumor          2
Name: sample_type, dtype: int64

In [15]:
tu_df.to_csv('sampletpy.csv')

In [16]:
fg_df= pd.read_csv('sampletpy.csv')

# Drop column 'file_id' from fg_df
fg_df.drop('file_id',axis=1, inplace=True)

data_tc= pd.read_csv('FinalData.csv')
data_tc = data_tc.set_index('gene_name').transpose()
data_tc.index.name = 'entity_submitter_id'
result = pd.merge(data_tc, fg_df, on='entity_submitter_id')

result = result.rename(columns={'entity_submitter_id':'gene'})
result = result.rename(columns={'sample_type':'label'})
print(result['label'].unique())
result = result.replace(to_replace='Solid Tissue Normal',value='0')
result = result.replace(to_replace='Primary Tumor',value='1')
result = result.replace(to_replace='Recurrent Tumor',value='1')
result.to_csv('TCGA_Labeled.csv',index=False)

['Primary Tumor' 'Solid Tissue Normal' 'Recurrent Tumor']


In [17]:
import pandas as pd
final_Df = pd.read_csv('TCGA_Labeled.csv')
final_Df['label'].value_counts()


1    535
0     59
Name: label, dtype: int64